In [ ]:
#The purpose of this code is to caluclate the distance between Brightest Cluster Galaxies and their host clusters. 
#Specfically, this code matches it calculates the distance between BCGs and their SZ center as well as the BCGs and their respective 
#host cluster Xray center.First, files containing the host cluster information are imported from the directory. 
#Then, files that will be written into for later use are defined. 



In [2]:
import numpy as np
import astropy
import regions
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.io import fits
from astropy import units as u 
from regions import read_ds9, write_ds9
from astropy.coordinates import SkyCoord
import glob, os
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion
from astropy.coordinates import SkyOffsetFrame
from astropy import cosmology 
import math
from astropy.table import Table, Column

In [4]:
#defining files in the directory to be used in the code
filename = 'C://Users/Janel/Desktop/urop_spt_clusters.txt'
filename1 = 'C://Users/Janel/Desktop/mbb_win_compat_2'
filename2 = 'C://Users/Janel/Desktop/BCG_xray_mod.txt'
#defining files to be written into 
ofile = 'C://Users/Janel/Desktop/All_separation_outputs.txt'   #master file with all of the values we calulated and the numbers they are derived from (old) 
outf1 = 'C://Users/Janel/Desktop/BCG-Xray_peak_separation.txt' #just the bcg-xray peak sep numbers 
outf2 = 'C://Users/Janel/Desktop/BCG-Xray_cent_separation.txt' #just the BCG-xray center sep numbers - although we wont be using this file 
#defining more files to be written intno 
coutfil = 'C://Users/Janel/Desktop/SZ_Xray_matched_clusters.txt' #matched files for writing
boutfil = 'C://Users/Janel/Desktop/all_BCG_coords.txt'#master file with all the values we calculated and the numbers they are derived from (new)
coutfits= 'C://Users/Janel/Desktop/SZ_Xray_matched_clusters.fits'#fits version of the above file (although we did not end up using it, here for back up in case the other text file crashes)
boutfits = 'C://Users/Janel/Desktop/all_BCG_coords.fits'#fits version of the above file ''''''''''
 
os.chdir(filename1) #creates a path for the file to be read into windows format?? 
cdata = ascii.read(filename)   # this reads the context of filename into a "Table" object which is apparently a thing in python
                               # Look up Data Tables (astropy.table) for how they work, see: # docs.astropy.org/en/stable/table/index.html#astropy-table
 
                               # using brackets lets you index columns by column                                
cnames1 = cdata['col1']        #cluster names 
szra = cdata['col2']           #SZ right ascension 
szdec = cdata['col3']          #SZ declination 
Nmem = cdata['col4']           #Number of cluster members 
z_cls = cdata['col5']          #redshift 
z_cl_err = cdata['col6']       #redshift error
vd_biw = cdata['col7']         #veolcity?
vd_err = cdata['col8']         #velocity error 
 
 
mdata = Table(names=('Name', 'Redshift', 'Redshift_Err','N_Members','vd_biw','vd_err','SZ_RA','SZ_Dec','Xpeak_RA','Xpeak_Dec','Xcen_RA','Xcen_Dec'), dtype=('U17','f8','f8','i8','f8','f8','f8','f8','f8','f8','f8','f8'))
                               #dtype: formatting based on amount of memory needed to process the numbers. 
                               #32 vs 64 bit = smaller vs larger numbers 
                               #cof1.write(cnames1[i],z_cls[i],z_cl_err[i],Nmem[i],vd_biw[i],vd_err[i],szra[i],szdec[i],c4.ra.deg[0],c4.dec.deg[0],c3.ra.deg[0],c3.dec.deg[0])
bdata = Table(names=('Name', 'Redshift', 'Redshift_Err','N_Members','vd_biw','vd_err','SZ_RA','SZ_Dec','Xpeak_RA','Xpeak_Dec','Xcen_RA','Xcen_Dec','BCG_RA','BCG_Dec','BCG_SZ_AngDist','BCG_Xp_AngDist','BCG_Xc_AngDist','SZ_Xray_AngDist'), dtype=('U17','f8','f8','i8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8','f8'))
                               #above are the headers for the masterfile 
                               # using brackets lets you index row by row
                               # using brackets lets you index row by row
 
 
f = open(filename,'r')         # opens urop_spt_clusters.txt file and 'r' reads it 
f2 = open(filename2,'r')       #opens up BCG_xray_mod.txt file and 'r' reads it 
 
ofil = open(ofile,'w')         #opens up ofile for writing 
outf = open(outf1, 'a')        #opens up outf1 for writing and appending values 
#cof1 = open(coutfil,'a')
bof1 = open(boutfil,'a')       #opens up boutfil for writing 
                               # print a header portion of the file with info about the columns:
hdrstring = '# Cluster_Name Cluster_z BCG_cnt  SZ-Xcen            BCG-Xcen            BCG-SZ         BCG-Xpeak'+'\n' #headers for seperations masterfile 
hdrstring2 = '# BCG-Xpeak'+'\n'#header for bcg_xraypeak seperations 
ofil.write(hdrstring)          #writes headers into file 
outf.write(hdrstring2)         #writes headers into file 
                               #of1 = open(outf1,'a')
                               #of2 = open(outf2,'a')
 
 
cos = astropy.cosmology.FlatLambdaCDM(H0 = 70, Om0 = 0.3, Tcmb0 = 2.725)  #defining cosmology 
 
      
                               #old cluster sky coordinate c1 = SkyCoord(cra, cdec, unit='deg', frame = 'fk5')
adj = cos.angular_diameter_distance(z_cls).value #distance that relates angular size to physical size in Mpc
 
xraycount = 0
 
for line in f2:
    xraycount = xraycount+1    #counts the number of 
    columns1 = line.split()    #splits file into columns 
    xray_name1 = columns1[0:1] #indexing row by row to get values 
    xray_peak_ra1 = columns1[1:2] #xray peak right ascension 
    xray_peak_dec1 = columns1[2:3] #xray peal declination 
    xray_centr_ra1 = columns1[3:4] 
    xray_centr_dec1 = columns1[4:5] 
    #print(xray_name1)
             
                                # we want to drop the outer loop that reads in, line by line, the urop_clusters_whatever.txt file, and instead 
                                # read that file in as a Table (w/ ascii.read). We then need to match the cluster name in the X-ray file to the 
                                # cluster names in the urop_clusters_whatever.txt file, and where there is a match grab the appropriate redshift 
                                # and SPT SZ coordinates 
    xray_name = '/n'.join(xray_name1)
    c4 = SkyCoord(xray_peak_ra1,xray_peak_dec1, unit = 'deg', frame ='fk5')
    c3 = SkyCoord(xray_centr_ra1,xray_centr_dec1, unit = 'deg',frame = 'fk5')
                                # Here is where we want to match xray_name with the cname array from the urop_spt_clusters.txt 
                                # file, and whatever index has that match, we use it to grab the relevant SPT SZ cluster data, 
                                # such as the redshift and the SZ coordinates (RA and Dec)
    for i in range(len(cdata)): #for the length of cdata
#    for line in cdata:
#        if xray_name in line:
#            cnames1 = line['col1']
#            szra = line['col2']
#            szdec = line['col3']
#            Nmem = line['col4']
#            z_cls = line['col5']
#            z_cl_err = line['col6']
#            vd_biw = line['col7']
#            vd_err = line['col8']
        if cnames1[i] == xray_name: #find matches within each of the files 
#           print(cnames1[i],z_cls[i],z_cl_err[i],Nmem[i],vd_biw[i],vd_err[i],szra[i],szdec[i],c4.ra.deg[0],c4.dec.deg[0],c3.ra.deg[0],c3.dec.deg[0])
            mdata.add_row((cnames1[i],z_cls[i],z_cl_err[i],Nmem[i],vd_biw[i],vd_err[i],szra[i],szdec[i],c4.ra.deg[0],c4.dec.deg[0],c3.ra.deg[0],c3.dec.deg[0]))
            #create rows for matching clusters with data 
            adj = cos.angular_diameter_distance(z_cls[i]).value     #Prints angular distance using matching redshift values in Mpc
         
            c1 = SkyCoord(szra[i],szdec[i],unit = 'deg', frame = 'fk5') #SZ right ascension and declination 
      
                                #the loop above reads the length of cdata and matacheds the cluster names to the xray names. 
                                #Relevant data matches print out under "if"
 
           
         
#            for file in glob.glob(xray_name +'*_bcgs*'): #reads bcg file in order according to the cnames1 file 
            for file in glob.glob(cnames1[i] +'*_bcgs*'): #reads bcg file in order according to the cnames1 file file 
                bcgreg = read_ds9(file)
                count3 = 0
                for region in bcgreg:
                    cen = region.center 
                    bra = cen.ra.deg
                    bdec = cen.dec.deg
                    c2 = SkyCoord(bra, bdec, unit='deg', frame = 'fk5') 
                     
                    sep1 = c1.separation(c3)  #SZ and XRAY CENTER 
                    nsep1 = sep1.rad
                    asep1 = sep1.arcsec
                    sin1 = math.sin(nsep1)
                    distance1 = np.multiply(sin1,adj)
                    ndistance1 = abs(np.multiply(distance1,1000))
             
             
                    sep2 = c2.separation(c3)  #BCG AND XRAY CENTER 
                    nsep2 = sep2.rad
                    asep2 = sep2.arcsec
                    sin2 = math.sin(nsep2)
                    distance2 = np.multiply(sin2,adj)
                    ndistance2 = abs(np.multiply(distance2,1000))
                     
                    sep3 = c2.separation(c1)  #BCG AND SZ
                    nsep3 = sep3.rad
                    asep3 = sep3.arcsec
                    sin3 = math.sin(nsep3)
                    distance3 = np.multiply(sin3,adj)
                    ndistance3 = abs(np.multiply(distance3,1000))
 
                     
                    sep4 = c2.separation(c4)  #BCG AND XRAY Peak
                    nsep4 = sep4.rad
                    asep4 = sep4.arcsec
                    sin4 = math.sin(nsep4)
                    distance4 = np.multiply(sin4,adj)
                    ndistance4 = abs(np.multiply(distance4,1000))
 
 
                    bdata.add_row((cnames1[i],z_cls[i],z_cl_err[i],Nmem[i],vd_biw[i],vd_err[i],szra[i],szdec[i],c4.ra.deg[0],c4.dec.deg[0],c3.ra.deg[0],c3.dec.deg[0],bra[0],bdec[0],ndistance3,ndistance4,ndistance2,ndistance1))
 
                    outstring =xray_name+'  '+str(z_cls[i])+'  '+str(count3)+'  '+str(ndistance1)+' '+str(ndistance2)+' '+str(ndistance3)+' '+str(ndistance4)+' \n'
                 
                    #ofil.write(outstring)
                    outstring_peak  = ''+str(ndistance4)+'\n'
                    #outf.write(outstring_peak)
                  
                    
 
 
#mdata.write(coutfil,format='ascii',overwrite=True)
#mdata.write(coutfits,format='fits',overwrite=True)
#print(bdata)
#bdata.write(boutfil,format='ascii',overwrite=True)
#bdata.write(boutfits,format='fits',overwrite=True)


      Name       Redshift Redshift_Err ...   BCG_Xc_AngDist    SZ_Xray_AngDist  
---------------- -------- ------------ ... ------------------ ------------------
SPT-CLJ0000-5748   0.7019       0.0006 ... 23.013851624389414  153.3999402912592
SPT-CLJ0013-4906   0.4075       0.0009 ...  107.5128947941855 29.524971146466118
SPT-CLJ0014-4952    0.752       0.0009 ... 246.25396917206646 123.02607453726105
SPT-CLJ0014-4952    0.752       0.0009 ... 363.32806670140843 123.02607453726105
SPT-CLJ0033-6326    0.599       0.0023 ... 52.557228095203826   96.7617794232311
SPT-CLJ0037-5047   1.0263       0.0012 ...  38.92082138701068 282.73033583757945
SPT-CLJ0040-4407   0.3498        0.001 ... 58.046919197662994 50.273254131274186
SPT-CLJ0102-4603   0.8405       0.0014 ... 126.99020192018143  89.37290409005836
SPT-CLJ0106-5943   0.3484       0.0011 ...  50.30973863312421 26.000913626912226
SPT-CLJ0123-4821    0.655       0.0018 ...  43.80060145843237 56.008641378828536
             ...      ...   